In [1]:
import pandas as pd
from pathlib import Path

In [10]:
# Read in data
data = Path("./data/cell_table_final.csv").resolve()
data = pd.read_csv(data)
data.head()

,fov,label,cell_size,X_centroid,Y_centroid,Eccentricity,Solidity,Extent,Orientation,cell_meta_cluster,...,region,region_name,refined_cell_type,final_cell_type,endocardial_annotation,exclude_annotation,artefact,distance_from_lumen,lumen_bin,size_filter
0,24h_86,1,619,23356.368336,2304.602585,0.529328,0.962675,0.764198,-0.617105,out_of_mask,...,4,background,out_of_mask,exclude,0,True,False,8915.498247,112.0,False
1,24h_86,2,727,28690.766162,2545.367263,0.821353,0.969333,0.677540,-1.067879,out_of_mask,...,4,background,out_of_mask,exclude,0,True,False,12092.505985,152.0,False
2,24h_86,3,1053,24556.710351,3095.472934,0.692851,0.901541,0.716327,1.303128,out_of_mask,...,4,background,out_of_mask,exclude,0,True,False,9028.624923,113.0,False
3,24h_86,4,554,24589.252708,3103.557762,0.827008,0.819527,0.586243,0.536782,out_of_mask,...,4,background,out_of_mask,exclude,0,True,False,9041.523157,114.0,False
4,24h_86,5,3219,14522.277415,4275.552656,0.492453,0.958036,0.752454,-1.437275,out_of_mask,...,4,background,out_of_mask,exclude,0,True,False,5593.401291,70.0,False


In [11]:
# How many FOVs do we have
print(f"We have {data['fov'].nunique()} FOVs")
print(data["fov"].unique())

We have 9 FOVs
['24h_86' '24h_83' 'Control_13' 'Control_12' '4h_97' '4h_96' '48h_79'
 '48h_76' 'Control_14']


In [16]:
# Show columns
data.columns

Index(['fov', 'label', 'cell_size', 'X_centroid', 'Y_centroid', 'Eccentricity',
       'Solidity', 'Extent', 'Orientation', 'cell_meta_cluster', 'timepoint',
       'region', 'region_name', 'refined_cell_type', 'final_cell_type',
       'endocardial_annotation', 'exclude_annotation', 'artefact',
       'distance_from_lumen', 'lumen_bin', 'size_filter'],
      dtype='object')

In [12]:
# Show columns
data["final_cell_type"].unique()

array(['exclude', 'Fibroblasts', 'Mono / Macros Ccr2+', 'Cardiomyocytes',
       'Neutrophils', 'Cardiomyocytes Ankrd1+', 'Endothelial cells',
       'Macrophages Trem2+', 'Smooth muscle cells', 'Macrophages Trem2-',
       'Other Leukocytes', 'Endocardial cells'], dtype=object)

In [13]:
# We need to exclude entries ['out_of_mask', 'background']  from refined_cell type and ['background', 'ignore', 'lumen']  from region_name
print(data.shape)
data = data[~data["final_cell_type"].isin(['exclude'])]
print(data.shape)
# data = data[~data["region_name"].isin(['background', 'ignore', 'lumen'])]
# print(data.shape)

(628365, 21)
(563180, 21)


In [17]:
# Iterate per FOV
for i, d in data.groupby("fov"):
    # Convert the refined_cell_type column to a one hot encoding
    ohe = pd.get_dummies(d, columns=["final_cell_type"], dtype=int)
    
    # Save the one hot encoding to a file
    ohe.to_csv(f"./data/ohe/{i}.csv", index=False)
# Read in the one hot encoded data